In [1]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import numpy as np

# Read in Data

In [2]:
%run "Data Collection and Cleaning.ipynb"

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 0 to 119
Data columns (total 54 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   abbreviation                           120 non-null    object 
 1   assists                                120 non-null    int64  
 2   blocks                                 120 non-null    int64  
 3   defensive_rebounds                     120 non-null    int64  
 4   field_goal_attempts                    120 non-null    int64  
 5   field_goal_percentage                  120 non-null    float64
 6   field_goals                            120 non-null    int64  
 7   free_throw_attempts                    120 non-null    int64  
 8   free_throw_percentage                  120 non-null    float64
 9   free_throws                            120 non-null    int64  
 10  games_played                           120 non-null    int64  
 11  minute

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [3]:
df_merged_train_data

,abbreviation,assists,blocks,defensive_rebounds,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempts,free_throw_percentage,free_throws,...,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,Season,W,L,W/L%,PS/G,PA/G,SRS
0,GSW,2491,555,2873,7140,0.495,3532,1850,0.788,1457,...,4578,0.557,2550,2017,58,24,0.707,113.5,107.5,5.79
1,HOU,2070,352,2746,7152,0.462,3305,2177,0.766,1667,...,3846,0.552,2124,2017,65,17,0.793,112.4,103.9,8.21
2,DEN,2077,323,2837,7194,0.469,3377,1986,0.774,1537,...,4829,0.519,2507,2017,46,36,0.561,110.0,108.5,1.57
3,CLE,1858,327,2821,6963,0.470,3275,1913,0.748,1431,...,4184,0.528,2208,2017,50,32,0.610,110.9,109.9,0.59
4,WAS,1956,336,2671,7137,0.475,3388,1812,0.784,1421,...,5107,0.515,2632,2017,43,39,0.524,106.6,106.0,0.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,MIA,1991,448,2879,7218,0.450,3251,1782,0.695,1238,...,4560,0.509,2323,2019,44,29,0.603,112.0,109.1,2.59
86,CHI,1796,351,2799,7205,0.453,3266,1695,0.783,1328,...,5082,0.496,2521,2019,22,43,0.338,106.8,109.9,-4.00
87,NYK,1646,422,2811,7241,0.433,3134,1956,0.759,1484,...,4820,0.479,2311,2019,21,45,0.318,105.8,112.3,-6.72
88,CLE,1698,195,2619,7184,0.444,3189,1694,0.792,1342,...,4796,0.488,2342,2019,19,46,0.292,106.9,114.8,-7.77


In [4]:
#create dataframe with the high correlation columns, and split up into train and test data
high_corr_df = df_merged_train_data.copy()
high_corr_df = high_corr_df[high_corr_win_cols]

train, test = high_corr_df[0:0], high_corr_df[0:0] 
for team in high_corr_df.abbreviation.unique():
    
    r_season = int(np.random.randint(2017, 2020))
#     print(team, r_season)
    
    temp_train = high_corr_df[(high_corr_df.Season != r_season) & (high_corr_df.abbreviation == team)]
    temp_test = high_corr_df[(high_corr_df.Season == r_season) & (high_corr_df.abbreviation == team)]
    
    train = train.append(temp_train)
    test = test.append(temp_test)
    
train = train.sample(train.shape[0])
test = test.sample(test.shape[0])

x_train = train[high_corr_win_cols].set_index(['abbreviation', 'Season']).drop("W", axis = 1)
y_train = train["W"].values.reshape(-1, 1)

x_test = test[high_corr_win_cols].set_index(['abbreviation', 'Season']).drop("W", axis = 1)
y_test = test["W"].values.reshape(-1, 1)


# y_data = high_corr_df["W"].values.reshape(-1, 1)
# x_data = high_corr_df[high_corr_win_cols].set_index(['abbreviation', 'Season']).drop('W', axis = 1)
# x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, random_state = 42)


In [5]:
#Train Linear Rregression Model
win_regr = LinearRegression().fit(x_train, y_train)

y_predict_train = win_regr.predict(x_train)
y_predict_test = win_regr.predict(x_test)

Since sklearn accuracy function is based on exact matches, it is tough to use the accuracy_score function since it will compare a decimal to an integer. So we will test in two different ways. The first test will be by rounding to the nearest integer, and comparing that result using the accuracy score. The seconcd test will be to get the rate of wins within X of the true value, where X is user defined.

Test #1

In [6]:
def rounded_accuracy(y_actual, y_pred):
    round_y_predict = []

    for y in y_pred:
        rounded_y = round(float(y))
        round_y_predict.append(rounded_y)
        
    round_y_predict_arr = np.array(round_y_predict)
    accuracy_sc = accuracy_score(y_actual, round_y_predict_arr)

    return accuracy_sc

In [7]:
train_accuracy = rounded_accuracy(y_train,y_predict_train)
print("Training Accuracy based on rounded: ",train_accuracy)

test_accuracy = rounded_accuracy(y_test,y_predict_test)
print("Test Accuracy based on rounded: ",test_accuracy)

Training Accuracy based on rounded:  0.016666666666666666
Test Accuracy based on rounded:  0.03333333333333333


Test #2

In [8]:
def calc_accuracy_rate(match_bool):
    true_count = 0
    total_matches = len(match_bool)
    for match in match_bool:
        if match:
            true_count += 1

    return true_count / total_matches

In [9]:
#Read linear Regression Model's Accuracy
def get_accuracy(num_wins_window, y_predict, y_actual):
    # if number of wins predicted is within 5 of the actual number of wins 
    # then this will be marked as a "success"
    # (for example if actual number of wins is 65, "correct window" is 60-70)
    # this can easily be adjusted based on the users desired level of accuracy

    round_y_predict = []

    for i,y in enumerate(y_predict):
        wins_diff = abs(y - y_actual[i])
        if wins_diff <= num_wins_window:
            match = True
        else:
            match = False
        round_y_predict.append(match)

        
    # round_y_predict_arr = np.array(round_y_predict)
    train_accuracy = calc_accuracy_rate(round_y_predict)

    return train_accuracy

In [10]:
train_accuracy_0 = get_accuracy(0,y_predict_train,y_train)
print("Exact Accuracy:",train_accuracy_0)

train_accuracy_1 = get_accuracy(1,y_predict_train,y_train)
print("Rate of correct wins within 1:", train_accuracy_1)

Exact Accuracy: 0.0
Rate of correct wins within 1: 0.08333333333333333


In [11]:
test_accuracy_0 = get_accuracy(0,y_predict_test,y_test)
print("Exact Accuracy:",test_accuracy_0)

test_accuracy_1 = get_accuracy(1,y_predict_test,y_test)
print("Rate of correct wins within 1:", test_accuracy_1)

test_accuracy_5 = get_accuracy(5,y_predict_test,y_test)
print("Rate of correct wins within 5:", test_accuracy_5)

Exact Accuracy: 0.0
Rate of correct wins within 1: 0.1
Rate of correct wins within 5: 0.3333333333333333
